# 【課題】腫瘍が良性か悪性かを予測する

まずは課題の説明欄に記載したURLから腫瘍の計測データが入ったzipファイルをダウンロードし、解凍してください。
表示された `breast_cancer_wisconsin_data.csv` ファイルを、Cloud9のワークスペース直下（このノートブックと同じディレクトリ）にアップロードします。

## モデルを決める

今回は、腫瘍が良性か悪性かを分類する機械学習プログラムを作ります。

そこで、レッスン本文でも利用した「サポートベクトル」の分類モデル（`SVC`）を使います。

## 学習に使うデータセットをインポートする

`breast_cancer_wisconsin_data.csv` ファイルのアップロードが完了した状態で、下記のコードを実行して、画像のデータセットを読み込んでください。

In [2]:
# breast_cancer_wisconsin_data.csv を取り込む（命令を追記すること）
import pandas as pd
breast_data = pd.read_csv("breast_cancer_wisconsin_data.csv")
# 先頭の5行のみ表示する（命令を追記すること）
breast_data.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


このCSVファイルには30以上の列があります。主要な列のみ、以下に概要を記載します。

- id：連番
- diagnosis："B"か"M"の文字が格納されている（"B"：良性、"M"：悪性）
- radius_mean：中心から外周までの平均距離（半径）
- texture_mean：グレースケール（色の濃さ）の平均値
- perimeter_mean：外周の平均の長さ

今回は `radius_mean`（半径）を説明変数、`diagnosis`（良性か悪性か）を目的変数として分類を行います。

データの前処理や分割を行う前に、`radius_mean`列を変数 `X` 、`diagnosis` 列を変数 `y` に格納してください。また、DataFrame形式から ndarray 形式に変換しましょう。

In [6]:
# X と y を作成する（命令を追記すること）
import numpy as np
X = np.array(breast_data.loc[:,["radius_mean"]])
y = np.array(breast_data.loc[:,["diagnosis"]])

## データの前処理を行う

さて、`y` には "B" もしくは "M" という文字データが入っています。数値化した方がコンピュータは学習しやすくなるので、データの前処理のひとつ「カテゴリ値の数値化」を実行しましょう。カテゴリ値の数値化をするには `LabelEncoder` というものを利用します。

[sklearn.preprocessing.LabelEncoder - scikit-learn 0.21 documentation](http://scikit-learn.org/0.21/modules/generated/sklearn.preprocessing.LabelEncoder.html)

※ `LabelEncoder` は、レッスン本文では登場しませんでしたので、以下のコードをそのまま実行いただければ大丈夫です。

In [7]:
# カテゴリ値の数値化
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(["B", "M"])                         # 良性：0, 悪性：1
y = le.transform(y.flatten())

# 数値化した状態を確認してみる
print(y)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 0 1 1 1 1 1 1 1 1 0 1 0 0 0 0 0 1 1 0 1 1 0 0 0 0 1 0 1 1 0 0 0 0 1 0 1 1
 0 1 0 1 1 0 0 0 1 1 0 1 1 1 0 0 0 1 0 0 1 1 0 0 0 1 1 0 0 0 0 1 0 0 1 0 0
 0 0 0 0 0 0 1 1 1 0 1 1 0 0 0 1 1 0 1 0 1 1 0 1 1 0 0 1 0 0 1 0 0 0 0 1 0
 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1 0 1 0 0 1 1 0 0 1 1 0 0 0 0 1 0 0 1 1 1 0 1
 0 1 0 0 0 1 0 0 1 1 0 1 1 1 1 0 1 1 1 0 1 0 1 0 0 1 0 1 1 1 1 0 0 1 1 0 0
 0 1 0 0 0 0 0 1 1 0 0 1 0 0 1 1 0 1 0 0 0 0 1 0 0 0 0 0 1 0 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 0 0 0 0 0 0 1 0 1 0 0 1 0 0 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 1 0 0 0 0 1 1 1 0 0
 0 0 1 0 1 0 1 0 0 0 1 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 1
 1 0 1 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 0 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0
 0 1 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 1 0 0 0 0 0 1 0 0
 1 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0
 0 0 0 0 0 0 1 0 1 0 0 1 

## 	データを訓練データとテストデータに分ける

`X` および `y` を訓練データとテストデータに分けましょう。その際、訓練データ8割、テストデータ2割としてください。

In [8]:
# 訓練データ8割、テストデータ2割に分割する（命令を追記すること）
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8, test_size = 0.2)

## モデルを作って学習する

最初の説明のとおり、`SVC` を使って分類モデルを作成します。また、作成したモデルに訓練データを渡して、学習を行います。

In [10]:
# SVCの分類モデルを作成する（命令を追記すること）
from sklearn.svm import SVC
classifier = SVC(kernel = "linear", gamma = "scale")


# 訓練データを分類モデルに設定する（命令を追記すること）
classifier.fit(X_train, y_train)

SVC(kernel='linear')

## 期待する性能が出たかを評価する

この分類モデルを使って、モデルの評価をましょう。

In [11]:
# テストデータを使って分類を実行する（命令を追記すること）
y_pred = classifier.predict(X_test)

まずは、予測値と実際の値をそのまま表示してください。

In [12]:
# 予測値を表示する（命令を追記すること）
print(y_pred)

[1 0 0 0 0 0 0 1 0 1 1 1 1 1 1 0 0 0 1 0 0 1 1 0 0 1 1 0 0 0 1 0 0 0 0 0 0
 0 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 1 0 1 1 0 0 1 0 1 1 1 0 0 0 0 1 0 1 0 0 0
 0 0 1 0 0 0 0 1 1 0 0 1 0 0 0 1 0 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0
 0 0 1]


In [13]:
# 実際の値を表示する（命令を追記すること）
print(y_test)

[1 0 1 0 0 1 0 1 0 1 0 1 1 1 1 1 0 0 1 0 0 1 0 0 0 1 1 0 0 0 1 0 0 1 1 1 1
 0 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 1 0 1 1 0 0 1 0 1 1 1 0 0 0 0 1 0 0 1 0 0
 0 0 1 0 0 0 0 1 1 0 0 1 0 0 0 1 0 0 1 0 1 1 1 0 0 0 0 0 0 0 1 0 0 1 0 0 0
 0 0 1]


予測の精度を混合行列で示したり、正答率を表示したりしてみます。

In [14]:
# 混同行列で集計結果を表示する（命令を追記すること）
from sklearn import metrics
print(metrics.confusion_matrix(y_test, y_pred))

[[68  3]
 [10 33]]


In [15]:
# 正答率を表示する（命令を追記すること）
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.87      0.96      0.91        71
           1       0.92      0.77      0.84        43

    accuracy                           0.89       114
   macro avg       0.89      0.86      0.87       114
weighted avg       0.89      0.89      0.88       114

